     _________   ___   __   _______   __________   ________   _______   ___    __   _______   _______   ______  
    |___   ___| |   \ |  | |       | |___    ___| |   __   | |       | |   \  |  | |   _   | |   _   | |       | 
        | |     |    \|  | |  _____|     |  |     |  |__|  | |  _____| |    \ |  | |  | |  | |  | |  | |    _  | 
        | |     |        | | |_____      |  |     |   __   | | |_____  |     \|  | |  | |  | |  | |  | |   |_| | 
        | |     |  |\    | |_____  |     |  |     |  |  |  | |_____  | |  |\     | |  |_|  | |  |_|  | |    ___| 
     ___| |___  |  | \   |  _____| |     |  |     |  |  |  |  _____| | |  | \    | |       | |       | |   |     
    |_________| |__|  \__| |_______|     |__|     |__|  |__| |_______| |__|  \___| |_______| |_______| |___|     

Dump a list of user's followers and accounts that they follow to Excel to create data for social network script development. 

Known limitations:
- Gets up to 20,000 followers or following accounts per interaction. You can change this, if you're looking into particularly popular accounts... But it'll take forever to return the data! Realistically, the more you try to pull out of a single account, the more likely you'll get flagged for suspicious activity and your account could be locked, they could ban you IP, etc. Buyer beware - That's on you, if it happens. I rarely try to scrape accounts with more than 2,000 followers/following.
- Does not get bio info, and a few other fields. That's a limitation due to the underlying library, "Instagram Scraper" (https://github.com/realsirjoe/instagram-scraper). I'm personally only half-satisfied with this; I do *NOT* trust that it's not somehow storing credentials, and it doesn't return all the data it purports to. 
- Only grabs public data, or private data if the account you're logged into has access to it.

To Do:
- Reassess the wait times, to ensure a long pause is occurring frequently enough. The Instagram-Scraper library's github has recommended limits
- Companion scripts to visualize and do additional analysis on the exported excel docs. These include a network diagram in NetworkX, Venn Diagrams and Pie charts in Matplotlib, etc

In [2]:

"""
Setup dependencies, define functions

You need to create a sec.py doc in the same folder you have this notebook in, 
to store your Instagram username and password. If you fork this, add your sec file to gitignore, DON'T commit it!
"""

from igramscraper.instagram import Instagram
from time import sleep
# from . import sec # Depending on your IDE, you might need to import the Sec file this way
import sec 
import re, random, os, time
import pandas as pd

def shortsleep():
    sleepytime = random.randint(1, 20)
    print(f"\tSleeping for {sleepytime} seconds...")
    time.sleep(sleepytime)

def longsleep():
    sleepytime = random.randint(20, 90)
    print(f"\tSleeping for {sleepytime} seconds...")
    time.sleep(sleepytime)

def buildOutput(response, username, option):
    df1 = pd.DataFrame()
    for user in response['accounts']:
        x = re.split("\n", str(user))
        acct = {"Id": x[2][4:],
                "Username": x[3][10:],
                "Full Name": x[4][11:],
                "Bio": x[5][5:],
                "Profile Pic Url": x[6][17:],
                "External url": x[7][14:],
                "Number of published posts": x[8][27:],
                "Number of followers": x[9][21:],
                "Number of follows": x[10][19:],
                "Is private": x[11][12:],
                "Is verified": x[12][13:]}
        df1 = df1.append(other=acct, ignore_index=True)
    df1 = df1.set_index("Username")
    df1 = df1.sort_index(ascending=True)
    # print(df1)
    df1.to_excel(f"{username}_{option}.xlsx")
    del df1

def getuser(username):
    try:
        print(f"{username}")
        account = instagram.get_account(username)
        try:
            print(f"\tGetting followers...")
            response = instagram.get_followers(account.identifier, 20000, 100, delayed=True)  # Get 2000 followers of 'kevin', 100 a time with random delay between requests
            option = "followers"
            buildOutput(response, username, option)
            
            shortsleep()
            
            print(f"\tGetting following...")
            response = instagram.get_following(account.identifier, 20000, 100, delayed=True)
            option = "following"
            buildOutput(response, username, option)

        except Exception as e:
            print(f"\t{username} broke, moving along")
            print(e)
            pass
    except:
        pass

def getlist(listofpeople):
    peepcount = 0
    for peep in listofpeople:
        peepcount += 1
        print(f"({peepcount}/{len(listofpeople)}) {time.strftime('%H%M %d%b%y')}")
        getuser(peep)
        if peepcount % 5 ==0:
            longsleep()
        else:
            shortsleep()
print("Setup complete!")

Setup complete!


In [ ]:
### This will log you into the account you supply credentials for
### You only have to do this once per session; it should cache your 
### credentials locally to keep the session active for a set amount of time
instagram = Instagram()
instagram.with_credentials(sec.user, sec.pw, os.path.join(os.getcwd(), 'cache'))
instagram.login(force=False,two_step_verificator=True)

### You can add proxies and/or set your User Agent String to make your session more or less protected.
### I run this with a VPN on, and get an email saying "We saw you logged in from a strange device" each time I run it
### DO NOT RUN THIS REPEATEDLY DURING YOUR SESSION - It's a Jupyter Notebook so you can just repeatedly run the cells below

sleep(2) # Delay to mimic user
print("Logged in!")

In [ ]:
### Add usernames you wanna work on to this list
peoplelist = []

In [ ]:
getlist(peoplelist)